In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

# training data
train = pd.read_csv('../housing/train.csv')

# determine missing values
total_missing = train.isnull().sum().sort_values(ascending=False)
missing_per = (train.isnull().sum()/len(train)).sort_values(ascending=False)
missing_data = pd.concat([total_missing, missing_per], axis=1, keys=['Total Missing', 'Percent'])
print(missing_data)

# delete columns with more than 10% missing data
train = train.drop((missing_data[missing_data['Percent'] > 0.1]).index,1)
train = train.drop("Id", axis=1)


               Total Missing   Percent
PoolQC                  1453  0.995205
MiscFeature             1406  0.963014
Alley                   1369  0.937671
Fence                   1179  0.807534
FireplaceQu              690  0.472603
LotFrontage              259  0.177397
GarageCond                81  0.055479
GarageType                81  0.055479
GarageYrBlt               81  0.055479
GarageFinish              81  0.055479
GarageQual                81  0.055479
BsmtExposure              38  0.026027
BsmtFinType2              38  0.026027
BsmtFinType1              37  0.025342
BsmtCond                  37  0.025342
BsmtQual                  37  0.025342
MasVnrArea                 8  0.005479
MasVnrType                 8  0.005479
Electrical                 1  0.000685
Utilities                  0  0.000000
YearRemodAdd               0  0.000000
MSSubClass                 0  0.000000
Foundation                 0  0.000000
ExterCond                  0  0.000000
ExterQual                

In [2]:
# encode some categorical variables from the ones remaining that are actually numerical variables (ordered encoding)

train = train.replace({"BsmtCond" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "BsmtExposure" : {"No" : 0, "Mn" : 1, "Av": 2, "Gd" : 3},
                       "BsmtFinType1" : {"No" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, "ALQ" : 5, "GLQ" : 6},
                       "BsmtFinType2" : {"No" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, "ALQ" : 5, "GLQ" : 6},
                       "BsmtQual" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5},
                       "ExterCond" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5},
                       "ExterQual" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5},
                       "Functional" : {"Sal" : 1, "Sev" : 2, "Maj2" : 3, "Maj1" : 4, "Mod": 5, "Min2" : 6, 
                                       "Min1" : 7, "Typ" : 8},
                       "GarageCond" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "GarageQual" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "HeatingQC" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "KitchenQual" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "LandSlope" : {"Sev" : 1, "Mod" : 2, "Gtl" : 3},
                       "LotShape" : {"IR3" : 1, "IR2" : 2, "IR1" : 3, "Reg" : 4},
                       "PavedDrive" : {"N" : 0, "P" : 1, "Y" : 2},
                       "Street" : {"Grvl" : 1, "Pave" : 2},
                       "Utilities" : {"ELO" : 1, "NoSeWa" : 2, "NoSewr" : 3, "AllPub" : 4}}
                     )

In [3]:
# change some numerical variables that are actually categorical variables
# MSubClass is the structure/dwelling of the house and MoSold is the month that the house was sold

train = train.replace({"MSSubClass" : {20 : "SC20", 30 : "SC30", 40 : "SC40", 45 : "SC45", 
                                       50 : "SC50", 60 : "SC60", 70 : "SC70", 75 : "SC75", 
                                       80 : "SC80", 85 : "SC85", 90 : "SC90", 120 : "SC120", 
                                       150 : "SC150", 160 : "SC160", 180 : "SC180", 190 : "SC190"},
                       "MoSold" : {1 : "Jan", 2 : "Feb", 3 : "Mar", 4 : "Apr", 5 : "May", 6 : "Jun",
                                   7 : "Jul", 8 : "Aug", 9 : "Sep", 10 : "Oct", 11 : "Nov", 12 : "Dec"}}
                     )

In [4]:
# determine correlation between numerical features and the response variable SalePrice

print("Correlation of numerical variables to SalePrice")
corr_table = train.corr()['SalePrice']
ftr = corr_table.index
cor_dict = {'Feature':ftr, 'Corr':corr_table}
corr_df = pd.DataFrame(cor_dict)
corr_df = corr_df[['Feature', 'Corr']]
corr_df = corr_df.sort_values(by='Corr', ascending = False)
print(corr_df)


Correlation of numerical variables to SalePrice
                     Feature      Corr
SalePrice          SalePrice  1.000000
OverallQual      OverallQual  0.790982
GrLivArea          GrLivArea  0.708624
ExterQual          ExterQual  0.682639
KitchenQual      KitchenQual  0.659600
BsmtQual            BsmtQual  0.644019
GarageCars        GarageCars  0.640409
GarageArea        GarageArea  0.623431
TotalBsmtSF      TotalBsmtSF  0.613581
1stFlrSF            1stFlrSF  0.605852
FullBath            FullBath  0.560664
TotRmsAbvGrd    TotRmsAbvGrd  0.533723
YearBuilt          YearBuilt  0.522897
YearRemodAdd    YearRemodAdd  0.507101
GarageYrBlt      GarageYrBlt  0.486362
MasVnrArea        MasVnrArea  0.477493
Fireplaces        Fireplaces  0.466929
HeatingQC          HeatingQC  0.427649
BsmtFinSF1        BsmtFinSF1  0.386420
BsmtExposure    BsmtExposure  0.352958
WoodDeckSF        WoodDeckSF  0.324413
2ndFlrSF            2ndFlrSF  0.319334
OpenPorchSF      OpenPorchSF  0.315856
HalfBath        

In [5]:
# drop the numerical features that have a correlation between +/- 5% with SalePrice
delete_features = pd.DataFrame(corr_df.loc[(corr_df['Corr'] < 0.05) & (corr_df['Corr'] > -0.05), 'Feature'])
train = train.drop(delete_features['Feature'], axis=1)

# drop OverallCond-has a -7.8% correlation with SalePrice, which doesn't make sense as it should be positive correlation
train = train.drop('OverallCond', axis = 1)

# drop LandSlope as correlation is only 5.1% and based on gut feeling, it doesn't seem important
train = train.drop('LandSlope', axis = 1)

In [6]:
cor = train.corr()['SalePrice']
cor = pd.DataFrame(cor)
cor = cor.sort_values(by = 'SalePrice', ascending = False)
#print(cor)
#np.shape(cor)

In [7]:
# one-hot encode the categorical features
train = pd.get_dummies(train)

In [8]:
#determine correlation of newly encoded categorical features with SalePrice
print("Correlation of numerical variables to SalePrice")
corr_table2 = train.corr()['SalePrice']
ftr2 = corr_table2.index
cor_dict2 = {'Feature':ftr2, 'Corr':corr_table2}
corr_df2 = pd.DataFrame(cor_dict2)
corr_df2 = corr_df2[['Feature', 'Corr']]
corr_df2 = corr_df2.sort_values(by='Corr', ascending = False)
print(corr_df2)
np.shape(corr_df2)

Correlation of numerical variables to SalePrice
                                     Feature      Corr
SalePrice                          SalePrice  1.000000
OverallQual                      OverallQual  0.790982
GrLivArea                          GrLivArea  0.708624
ExterQual                          ExterQual  0.682639
KitchenQual                      KitchenQual  0.659600
BsmtQual                            BsmtQual  0.644019
GarageCars                        GarageCars  0.640409
GarageArea                        GarageArea  0.623431
TotalBsmtSF                      TotalBsmtSF  0.613581
1stFlrSF                            1stFlrSF  0.605852
FullBath                            FullBath  0.560664
TotRmsAbvGrd                    TotRmsAbvGrd  0.533723
YearBuilt                          YearBuilt  0.522897
YearRemodAdd                    YearRemodAdd  0.507101
Foundation_PConc            Foundation_PConc  0.497734
GarageYrBlt                      GarageYrBlt  0.486362
MasVnrArea       

(227, 2)

In [9]:
# drop the categorical features that have a correlation between +/- 5% with SalePrice
delete_features2 = pd.DataFrame(corr_df2.loc[(corr_df2['Corr'] < 0.05) & (corr_df2['Corr'] > -0.05), 'Feature'])
train = train.drop(delete_features2['Feature'], axis=1)
np.shape(train)

(1460, 137)

In [10]:
# create dataframe for training x variables
train_X = train

# fill in the remaining missing values with 0 
train_X = train_X.fillna(0)

# df for training y variables
train_y = train_X.SalePrice
train_X = train_X.drop(['SalePrice'], axis=1)

# fit multiple linear regression model
reg = LinearRegression().fit(train_X, train_y)
reg.score(train_X, train_y)

0.88386906144121502

In [11]:
# test data
test = pd.read_csv('../housing/test.csv')

# determine missing values in test data
total_missing1 = test.isnull().sum().sort_values(ascending=False)
missing_per1 = (test.isnull().sum()/len(test)).sort_values(ascending=False)
missing_data1 = pd.concat([total_missing1, missing_per1], axis=1, keys=['Total Missing', 'Percent'])
#print(missing_data)

# delete columns with more than 10% missing data in test dat 
test = test.drop((missing_data1[missing_data1['Percent'] > 0.1]).index,1)

In [12]:
# encode some categorical variables from the ones remaining that are actually numerical variables (ordered encoding)

test = test.replace({"BsmtCond" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "BsmtExposure" : {"No" : 0, "Mn" : 1, "Av": 2, "Gd" : 3},
                       "BsmtFinType1" : {"No" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, "ALQ" : 5, "GLQ" : 6},
                       "BsmtFinType2" : {"No" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, "ALQ" : 5, "GLQ" : 6},
                       "BsmtQual" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5},
                       "ExterCond" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5},
                       "ExterQual" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5},
                       "Functional" : {"Sal" : 1, "Sev" : 2, "Maj2" : 3, "Maj1" : 4, "Mod": 5, "Min2" : 6, 
                                       "Min1" : 7, "Typ" : 8},
                       "GarageCond" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "GarageQual" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "HeatingQC" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "KitchenQual" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "LandSlope" : {"Sev" : 1, "Mod" : 2, "Gtl" : 3},
                       "LotShape" : {"IR3" : 1, "IR2" : 2, "IR1" : 3, "Reg" : 4},
                       "PavedDrive" : {"N" : 0, "P" : 1, "Y" : 2},
                       "Street" : {"Grvl" : 1, "Pave" : 2},
                       "Utilities" : {"ELO" : 1, "NoSeWa" : 2, "NoSewr" : 3, "AllPub" : 4}}
                     )

In [13]:
# change some numerical variables that are actually categorical variables
# MSubClass is the structure/dwelling of the house and MoSold is the month that the house was sold

test = test.replace({"MSSubClass" : {20 : "SC20", 30 : "SC30", 40 : "SC40", 45 : "SC45", 
                                       50 : "SC50", 60 : "SC60", 70 : "SC70", 75 : "SC75", 
                                       80 : "SC80", 85 : "SC85", 90 : "SC90", 120 : "SC120", 
                                       150 : "SC150", 160 : "SC160", 180 : "SC180", 190 : "SC190"},
                       "MoSold" : {1 : "Jan", 2 : "Feb", 3 : "Mar", 4 : "Apr", 5 : "May", 6 : "Jun",
                                   7 : "Jul", 8 : "Aug", 9 : "Sep", 10 : "Oct", 11 : "Nov", 12 : "Dec"}}
                     )

In [14]:
# one-hot encoding for categorical features in test data
test = pd.get_dummies(test)
np.shape(test)

(1459, 226)

In [16]:
# delete features in test data that are not in training data
delete_test_features = []

test_cols = test.columns
train_cols = train.columns

for i in range(0,len(test_cols)):
    if (test_cols[i] not in train_cols):
        delete_test_features.append(test_cols[i])
        
test = test.drop(delete_test_features, axis=1)

In [18]:
# fill in missing values with 0
test=test.fillna(0)

# predict house prices in test data
result = reg.predict(test)
#print(result)

In [270]:
# output predictions to csv for kaggle submission
df = pd.DataFrame(data = result)
df.to_csv("final_prices.csv", sep=',', encoding='utf-8')